In [ ]:
import os
import json
import shutil
from tqdm import tqdm
import pickle as pk
import pandas as pd
import re
import copy
from pprint import pprint

In [ ]:
adr_kr_R = ""
adr_kr_NR = ""

for file in tqdm(os.listdir(adr_kr_R)):
    shutil.move(adr_kr_R+file,adr_kr_NR)

In [ ]:
def tweetElements(tweet):
    rtFlag = True if "retweeted_status" in tweet.keys() else False
    qtFlag = True if "quoted_status" in tweet.keys() else False 

    if rtFlag == False and qtFlag == False:
        if tweet["truncated"] == True:
            text = copy.deepcopy(tweet["extended_tweet"]["full_text"])
            entities = copy.deepcopy(tweet["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(tweet["text"])
            entities = copy.deepcopy(tweet["entities"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user]
    
    elif rtFlag == True and qtFlag == False:
        rt = copy.deepcopy(tweet["retweeted_status"])
        if rt["truncated"] == True:
            text = copy.deepcopy(rt["extended_tweet"]["full_text"])
            entities = copy.deepcopy(rt["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(rt["text"])
            entities = copy.deepcopy(rt["entities"])
        rtUser = copy.deepcopy(rt["user"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user, rtUser]
    
    elif rtFlag == False and qtFlag == True:
        if tweet["truncated"] == True:
            text = copy.deepcopy(tweet["extended_tweet"]["full_text"])
            entities = copy.deepcopy(tweet["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(tweet["text"])
            entities = copy.deepcopy(tweet["entities"])
        user = copy.deepcopy(tweet["user"])
        qt = copy.deepcopy(tweet["quoted_status"])
        if qt["truncated"] == True:
            qtText = copy.deepcopy(qt["extended_tweet"]["full_text"])
            qtEntities = copy.deepcopy(qt["extended_tweet"]["entities"])
        else:
            qtText = copy.deepcopy(qt["text"])
            qtEntities = copy.deepcopy(qt["entities"])
        qtUser = copy.deepcopy(qt["user"])
        return [rtFlag, qtFlag, text, entities, user, qtText, qtEntities, qtUser]        
        
    elif rtFlag == True and qtFlag == True:
        rt = copy.deepcopy(tweet["retweeted_status"])
        qt = copy.deepcopy(tweet["quoted_status"])        
        if rt["truncated"] == True:
            text = copy.deepcopy(rt["extended_tweet"]["full_text"])
            entities = copy.deepcopy(rt["extended_tweet"]["entities"])
        else:
            text = copy.deepcopy(rt["text"])
            entities = copy.deepcopy(rt["entities"])
        rtUser = copy.deepcopy(rt["user"])        
        if qt["truncated"] == True:
            qtText = copy.deepcopy(qt["extended_tweet"]["full_text"])
            qtEntities = copy.deepcopy(qt["extended_tweet"]["entities"])
        else:
            qtText = copy.deepcopy(qt["text"])
            qtEntities = copy.deepcopy(qt["entities"])
        qtUser = copy.deepcopy(qt["user"])
        user = copy.deepcopy(tweet["user"])
        return [rtFlag, qtFlag, text, entities, user, qtText, qtEntities, qtUser, rtUser]        

In [ ]:
# This loop, iterates over all the collected tweets and copy the ones that are relevant to the rumours
for file in tqdm(os.listdir(adr_kr_NR)):
    tweet = json.load(open(adr_kr_NR+file))

    elements = tweetElements(tweet)
    rtFlag = copy.deepcopy(elements[0])
    qtFlag = copy.deepcopy(elements[1]) 

    
    if rtFlag == False and qtFlag == False:
        text, entities, user = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4])
    elif rtFlag == True and qtFlag == False:
        text, entities, user, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5])
    elif rtFlag == False and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7])       
    elif rtFlag == True and qtFlag == True:
        text, entities, user, qtText, qtEntities, qtUser, rtUser = copy.deepcopy(elements[2]), copy.deepcopy(elements[3]), copy.deepcopy(elements[4]), copy.deepcopy(elements[5]), copy.deepcopy(elements[6]), copy.deepcopy(elements[7]), copy.deepcopy(elements[8])
                    
    tweetText = text.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ")
    qtTweetText = qtText.lower().strip().replace("#"," ").replace("@", " ").replace("  ", " ").replace("\t", " ") if qtFlag else " "
    
    flag = False
    if re.search('.*church.*only.*christ.*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*navy.*helicopter.*number', tweetText, re.DOTALL) != None or re.search('.*helicopter.*phone.*', tweetText, re.DOTALL) != None or \
        re.search('.*navy.*phone.*', tweetText, re.DOTALL) != None or re.search('.*8281292702.*', tweetText, re.DOTALL) != None :
        flag = True
    elif re.search('.*cancel.*onam.*', tweetText, re.DOTALL) != None or re.search('.*onam.*cancel.*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*luxury.*car.*', tweetText, re.DOTALL) != None or \
        re.search('.*misuse.*(donation|relief).*fund.*', tweetText, re.DOTALL) != None or re.search('.*fascist.*anti.*bharat.*left.*', tweetText, re.DOTALL) != None or \
        re.search('.*break.*india.*proj.*', tweetText, re.DOTALL) != None or re.search('.*donat.*misuse.*', tweetText, re.DOTALL) != None or \
        re.search('.*relief.*misuse.*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*imposter.*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*(crack|burst|collapse|leak|break).*dam(s|) .*', tweetText, re.DOTALL) != None or re.search('.*dam(s|) .*(crack|burst|collapse|leak|break).*', tweetText, re.DOTALL) != None :
        flag = True
    elif re.search('.*renault.*', tweetText, re.DOTALL) != None or re.search('.*(carpark|car park).*flood.*', tweetText, re.DOTALL) != None or re.search('.*flood.*(carpark|car park).*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*kseb.*shut down.*', tweetText, re.DOTALL) != None or re.search('.*no electricity.*', tweetText, re.DOTALL) != None or re.search('.*power.*(blackout|cut).*', tweetText, re.DOTALL) != None:
        flag = True
    elif (re.search('.*(christiano|cristiano|ronaldo|cr7).*', tweetText, re.DOTALL) != None) and (re.search('.*(seventy seven|77).*(crore|cr).*', tweetText, re.DOTALL) != None or \
        re.search('.*(eleven|11).*million.*', tweetText, re.DOTALL) != None) :
        flag = True
    elif re.search('.*shah( |-)?rukh( |-)?khan.*', tweetText, re.DOTALL) != None and  re.search('.*(five|5) ?(crore|cr)*', tweetText, re.DOTALL) != None:
        flag = True
    elif re.search('.*united.*arab.*emirates.*', tweetText, re.DOTALL) != None or re.search('.*700.*(crore|cr).*', tweetText, re.DOTALL) != None or \
        re.search('.*(hundred|100)-?million.*', tweetText, re.DOTALL) != None or re.search('.*yusuff.*ali.*', tweetText, re.DOTALL) != None or re.search('.*(uae |emirates).*', tweetText, re.DOTALL) != None :
        flag = True
    elif re.search('.*(rumor|rumour|hoax|conspirac|fake-?news|fake-?info| myth | myths |disinfo|misinfo).*', tweetText, re.DOTALL) != None :
        flag = True

    #exceptions    
    if re.search('.*yusuff.*ali.*10.*(crore|cr).*', tweetText, re.DOTALL) != None or re.search('.*uae*10.*(crore|cr).*', tweetText, re.DOTALL) != None or re.search('.*yusuff.*ali.*50 million.*', tweetText, re.DOTALL) != None or\
       re.search('.*shamsher vayali.*', tweetText, re.DOTALL) != None or re.search('.*alain uae.*', tweetText, re.DOTALL) != None:                                                                                                                         
        exceptionFlag = False
    else:
        exceptionFlag = True                                                                                                                                      

    qflag = False
    if qtFlag:
        if re.search('.*church.*only.*christ.*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*navy.*helicopter.*number', qtTweetText, re.DOTALL) != None or re.search('.*helicopter.*phone.*', qtTweetText, re.DOTALL) != None or \
            re.search('.*navy.*phone.*', qtTweetText, re.DOTALL) != None or re.search('.*8281292702.*', qtTweetText, re.DOTALL) != None :
            qflag = True
        elif re.search('.*cancel.*onam.*', qtTweetText, re.DOTALL) != None or re.search('.*onam.*cancel.*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*luxury.*car.*', qtTweetText, re.DOTALL) != None or \
            re.search('.*misuse.*(donation|relief).*fund.*', qtTweetText, re.DOTALL) != None or re.search('.*fascist.*anti.*bharat.*left.*', qtTweetText, re.DOTALL) != None or \
            re.search('.*break.*india.*proj.*', qtTweetText, re.DOTALL) != None or re.search('.*donat.*misuse.*', qtTweetText, re.DOTALL) != None or \
            re.search('.*relief.*misuse.*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*imposter.*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*(crack|burst|collapse|leak|break).*dam(s|) .*', qtTweetText, re.DOTALL) != None or re.search('.*dam(s|).*(crack|burst|collapse|leak|break).*', qtTweetText, re.DOTALL) != None :
            qflag = True
        elif re.search('.*renault.*', qtTweetText, re.DOTALL) != None or re.search('.*(carpark|car park).*flood.*', qtTweetText, re.DOTALL) != None or re.search('.*flood.*(carpark|car park).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*kseb.*shut down.*', qtTweetText, re.DOTALL) != None or re.search('.*no electricity.*', qtTweetText, re.DOTALL) != None or re.search('.*power.*(blackout|cut).*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif (re.search('.*(christiano|cristiano|ronaldo|cr7).*', qtTweetText, re.DOTALL) != None) and (re.search('.*(seventy seven|77).*(crore|cr).*', qtTweetText, re.DOTALL) != None or \
            re.search('.*(eleven|11).*million.*', qtTweetText, re.DOTALL) != None) :
            qflag = True
        elif re.search('.*shah( |-)?rukh( |-)?khan.*', qtTweetText, re.DOTALL) != None and  re.search('.*(five|5) ?(crore|cr)*', qtTweetText, re.DOTALL) != None:
            qflag = True
        elif re.search('.*united.*arab.*emirates.*', qtTweetText, re.DOTALL) != None or re.search('.*700.*(crore|cr).*', qtTweetText, re.DOTALL) != None or \
            re.search('.*(hundred|100)-?million.*', qtTweetText, re.DOTALL) != None or re.search('.*yusuff.*ali.*', qtTweetText, re.DOTALL) != None or re.search('.*(uae |emirates).*', qtTweetText, re.DOTALL) != None :
            qflag = True
        elif re.search('.*(rumor|rumour|hoax|conspirac|fake-?news|fake-?info| myth | myths |disinfo|misinfo).*', qtTweetText, re.DOTALL) != None :
            qflag = True        
        
        #exception
        if re.search('.*yusuff.*ali.*10.*(crore|cr).*', qtTweetText, re.DOTALL) != None or re.search('.*uae*10.*(crore|cr).*', qtTweetText, re.DOTALL) != None or re.search('.*yusuff.*ali.*50 million.*', qtTweetText, re.DOTALL) != None or\
           re.search('.*shamsher vayali.*', qtTweetText, re.DOTALL) != None or re.search('.*alain uae.*', qtTweetText, re.DOTALL) != None:  
            exceptionFlag = False
        else:
            exceptionFlag = True

    finalFlag = (flag or qflag) and exceptionFlag
        
    if finalFlag:
        shutil.move(adr_kr_NR + file, adr_kr_R)